In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
import warnings
from sklearn.preprocessing import LabelEncoder
import gc
import os
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization

plt.style.use('seaborn')
sns.set(font_scale=1)
pd.set_option('display.max_columns', 500)

In [2]:
train = pd.read_csv('train.csv')
label = pd.read_csv('train_label.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('submission.csv')

# 数据探索

In [3]:
df_source = train.merge(label,on='ID',how='left')

# 特征工程

In [4]:
test['label'] = -1
train = train.merge(label,on='ID',how='left')
data = pd.concat([train, test])

In [5]:
def get_time_fe(df):
    df['day'] = df.date.apply(lambda x:int(x[8:10]))

    df['hour'] = df.date.apply(lambda x:int(x[11:13]))
    
#     df['d_h'] = df['day'].astype('int')*24+df['hour'].astype('int')
    
#     result = df[df['label']!=-1].groupby(['d_h']).label.agg(['count','sum']).reset_index()
    
#     result['click_prob'] = (result['sum'].astype('int')/result['count'].astype('int'))
    
#     df = df.merge(result,on=['d_h'],how = 'left')
    
#     del result['count'],result['sum'],result['d_h']
    
    return df

In [6]:
# szy = get_time_fe(data)
# result = szy[szy['label']!=-1].groupby(['hour']).label.agg(['count','sum']).reset_index()
# result['click_prob'] = (result['sum'].astype('int')/result['count'].astype('int'))

In [7]:
#对时间分箱
def getSeg(x):
    if x >=0 and x<= 3:
        return 1
    elif x>=4 and x<=12:
        return 2
    elif x>=13 and x<=18:
        return 3
    elif x>=19 and x<=23:
        return 1

In [8]:
# hourDF = train_df.groupby(['hour', 'label'])['hour'].count().unstack('label').fillna(0)
# hourDF[[0,1]].plot(kind='bar', stacked=True);

In [9]:
#count统计特征
cross_feature = []
def get_cross_fe(df):
    first_feature = [ 'B2', 'B3']
    second_feature = ['C1','C2','C3','D1','A1','A2','A3']
    for feat_1 in first_feature:
        for feat_2 in second_feature:
            col_name = "cross_" + feat_1 + "_and_" + feat_2
            cross_feature.append(col_name)
            df[col_name] = df[feat_1].astype(str).values + '_' + df[feat_2].astype(str).values
    return df

In [10]:
#获取nunique特征
def get_nunique_1_fe(df):
    adid_nuq = [ 'hour','E1','E14','B2','B3']
    for feat in adid_nuq:
        gp1 = df.groupby('A2')[feat].nunique().reset_index().rename(columns={feat: "A2_%s_nuq_num" % feat})
        gp2 = df.groupby(feat)['A2'].nunique().reset_index().rename(columns={'A2': "%s_A2_nuq_num" % feat})
        df = pd.merge(df, gp1, how='left', on=['A2'])
        df = pd.merge(df, gp2, how='left', on=[feat])
    return df
def get_nunique_2_fe(df):
    adid_nuq = [ 'E1','E14']
    for feat in adid_nuq:
        gp1 = df.groupby('hour')[feat].nunique().reset_index().rename(columns={feat: "hour_%s_nuq_num" % feat})
        gp2 = df.groupby(feat)['hour'].nunique().reset_index().rename(columns={'hour': "%s_hour_nuq_num" % feat})
        df = pd.merge(df, gp1, how='left', on=['hour'])
        df = pd.merge(df, gp2, how='left', on=[feat])
    return df

# def get_nunique_3_fe(df):
#     adid_nuq = ['B2','B3']
#     for feat in adid_nuq:
#         gp1 = df.groupby('A3')[feat].nunique().reset_index().rename(columns={feat: "A3_%s_nuq_num" % feat})
#         gp2 = df.groupby(feat)['A3'].nunique().reset_index().rename(columns={'A3': "%s_A3_nuq_num" % feat})
#         df = pd.merge(df, gp1, how='left', on=['A3'])
#         df = pd.merge(df, gp2, how='left', on=[feat])
#     return df

def get_nunique_4_fe(df):
    adid_nuq = [ 'B2','B3']
    for feat in adid_nuq:
        gp1 = df.groupby('A1')[feat].nunique().reset_index().rename(columns={feat: "A1_%s_nuq_num" % feat})
        gp2 = df.groupby(feat)['A1'].nunique().reset_index().rename(columns={'A1': "%s_A1_nuq_num" % feat})
        df = pd.merge(df, gp1, how='left', on=['A1'])
        df = pd.merge(df, gp2, how='left', on=[feat])
    return df

In [11]:
data = get_time_fe(data)
# data['hour_seg'] = data['hour'].apply(lambda x: getSeg(x))
data = get_cross_fe(data)
data = get_nunique_1_fe(data)
data = get_nunique_2_fe(data)
# data = get_nunique_3_fe(data)
# data = get_nunique_4_fe(data)

In [12]:
len(cross_feature)

14

In [13]:
#labelencoder()
cate_feature = ['A1','A2','A3','B1','B2','B3','C1','C2','C3','E2','E3','E5','E7','E9','E10','E13','E16','E17','E19','E21','E22']
# cross_feature = cross_feature[:15]
cate_features = cate_feature+cross_feature
for item in cate_features:
    data[item] = LabelEncoder().fit_transform(data[item])

In [14]:
def feature_count(data, features=[]):
    new_feature = 'count'
    for i in features:
        new_feature += '_' + i
    try:
        del data[new_feature]
    except:
        pass
    temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
    data = data.merge(temp, 'left', on=features)
    return data

for i in cross_feature:
    n = data[i].nunique()
    if n > 5:
        data = feature_count(data, [i])
    else:
        print(i, ':', n)

In [15]:
#ratio：类别偏好的ratio比例特征
label_feature =[ 'A2', 'A3','hour']
data_temp = data[label_feature]
df_feature = pd.DataFrame()
data_temp['cnt'] = 1
print('Begin ratio clcik...')
col_type = label_feature.copy()
n = len(col_type)
for i in range(n):
    col_name = "ratio_click_of_" + col_type[i]
    df_feature[col_name] = (
                    data_temp[col_type[i]].map(data_temp[col_type[i]].value_counts()) / len(data) * 100).astype(int)            
data = pd.concat([data, df_feature], axis=1)
print('The end')

Begin ratio clcik...
The end


In [16]:
train_df = data[data['label'] != -1]
test_df = data[data['label'] == -1]

In [17]:
# grouped_df = train_df.groupby(["day", "hour"])["label"].aggregate("mean").reset_index()
# grouped_df = grouped_df.pivot('day', 'hour', 'label')
# plt.figure(figsize=(12,6))
# sns.heatmap(grouped_df)
# plt.title("CVR of Day Vs Hour")
# plt.show()

In [18]:
# sns.stripplot(train_df["label"],train_df["A3"],jitter=True,order=None)
# plt.title("click Vs creative_height");

# 删除不需要的字段

In [19]:
## get train feature
del_feature = ['ID','day','date','label','D2']+cross_feature
features = [i for i in train_df.columns if i not in del_feature]

In [20]:
train_x = train_df[features]
train_y = train_df['label'].values
test = test_df[features]

# 参数训练

In [21]:
data = lgb.Dataset(train_x,train_y)

def lgb_cv(num_leaves,learning_rate,max_bin, min_data_in_leaf,feature_fraction,bagging_fraction,subsample,min_split_gain,min_child_weight,min_child_samples,reg_alpha,reg_lambda,num_iterations=2000):
        
        params = {'objective':'binary','num_iterations': num_iterations, 'early_stopping_round':50, 'metric':{'binary_logloss', 'auc'}}
        params["num_leaves"] = int(round(num_leaves))
        params["learning_rate"] = learning_rate
        params["max_bin"] = int(round(max_bin))
        params["min_data_in_leaf"] = int(round(min_data_in_leaf))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['subsample'] = max(min(subsample, 1), 0)

        
        params['min_split_gain'] = min_split_gain      
        
        params['min_child_weight'] = int(round(min_child_weight))
        params['min_child_samples'] = int(round(min_child_samples))
        
        params['reg_alpha'] = int(round(reg_alpha))
        params['reg_lambda'] = int(round(reg_lambda))

        
        cv_result = lgb.cv(params, data, nfold=10, seed=2, stratified=True, verbose_eval =50)
        return -(min(cv_result['auc-mean']))

lgb_bo = BayesianOptimization(
        lgb_cv,
        {'num_leaves': (16, 128),
         'learning_rate': (0.01, 0.4),
         'max_bin': (300, 550),
         'min_data_in_leaf': (1, 100),    
        'feature_fraction': (0.05, 1),
        'bagging_fraction': (0.5, 1),
        'subsample': (0.5, 1),

         'min_split_gain': (0, 5),
         'min_child_weight': (0,20),
         'min_child_samples':(0,100),
         
          'reg_alpha':(0,20),
         'reg_lambda':(0,20)})

lgb_bo.maximize(init_points=21,n_iter=90) #init_points表示初始点，n_iter代表迭代次数（即采样数）
print (lgb_bo.max)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | min_ch... | min_ch... | min_da... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[50]	cv_agg's binary_logloss: 0.41087 + 0.0027416	cv_agg's auc: 0.723333 + 0.00695153
|  1        | -0.6835   |  0.5555   |  0.2145   |  0.3384   |  319.9    |  0.8681   |  10.61    |  96.49    |  1.297    |  72.27    |  3.629    |  6.804    |  0.7613   |
[50]	cv_agg's binary_logloss: 0.412195 + 0.00256226	cv_agg's auc: 0.720692 + 0.00639282
|  2        | -0.7007   |  0.6891   |  0.9806   |  0.2477   |  306.8    |  50.94    |  19.06    |  19.64    |  1.751    |  61.89    |  6.264    |  17.09    |  0.9938   |
[50]	cv_agg's binary_logloss: 0.411759 + 0.00404343	cv_agg's auc: 0.722016 + 0.00877916
|  3        | -0.7001   |  0.6912   |  0.3716   |  0.3766 

[50]	cv_agg's binary_logloss: 0.412791 + 0.00195692	cv_agg's auc: 0.720275 + 0.00558887
[100]	cv_agg's binary_logloss: 0.412789 + 0.0019606	cv_agg's auc: 0.720275 + 0.00561827
|  20       | -0.6922   |  0.7436   |  0.2786   |  0.1438   |  392.5    |  72.63    |  11.1     |  32.06    |  2.278    |  30.16    |  7.048    |  7.065    |  0.9324   |
[50]	cv_agg's binary_logloss: 0.414386 + 0.00187992	cv_agg's auc: 0.715659 + 0.00520505
|  21       | -0.674    |  0.7477   |  0.3493   |  0.2669   |  490.9    |  54.53    |  4.131    |  32.27    |  1.346    |  18.38    |  18.03    |  16.57    |  0.91     |
[50]	cv_agg's binary_logloss: 0.41831 + 0.00155022	cv_agg's auc: 0.706952 + 0.0046094
|  22       | -0.677    |  0.8764   |  0.5923   |  0.1976   |  539.6    |  93.96    |  13.94    |  7.738    |  4.936    |  17.07    |  17.14    |  16.72    |  0.5748   |
[50]	cv_agg's binary_logloss: 0.412931 + 0.00208455	cv_agg's auc: 0.719195 + 0.00571219
|  23       | -0.6988   |  0.7322   |  0.6034   |  0

[50]	cv_agg's binary_logloss: 0.442263 + 0.000573902	cv_agg's auc: 0.692234 + 0.00554087
[100]	cv_agg's binary_logloss: 0.434383 + 0.00087917	cv_agg's auc: 0.695029 + 0.00561555
[150]	cv_agg's binary_logloss: 0.429606 + 0.00104876	cv_agg's auc: 0.697906 + 0.00553622
[200]	cv_agg's binary_logloss: 0.426517 + 0.00117537	cv_agg's auc: 0.700255 + 0.00548947
[250]	cv_agg's binary_logloss: 0.424493 + 0.0012818	cv_agg's auc: 0.701461 + 0.00559326
[300]	cv_agg's binary_logloss: 0.423155 + 0.00135553	cv_agg's auc: 0.702607 + 0.00577773
[350]	cv_agg's binary_logloss: 0.422103 + 0.00143507	cv_agg's auc: 0.703751 + 0.00580785
[400]	cv_agg's binary_logloss: 0.42142 + 0.00147161	cv_agg's auc: 0.7045 + 0.00575904
[450]	cv_agg's binary_logloss: 0.42099 + 0.00149405	cv_agg's auc: 0.704869 + 0.00573519
[500]	cv_agg's binary_logloss: 0.420587 + 0.0015208	cv_agg's auc: 0.705269 + 0.00568289
[550]	cv_agg's binary_logloss: 0.420276 + 0.00155563	cv_agg's auc: 0.705532 + 0.00570784
[600]	cv_agg's binary_loglo

[700]	cv_agg's binary_logloss: 0.41918 + 0.00163673	cv_agg's auc: 0.707458 + 0.00585786
[750]	cv_agg's binary_logloss: 0.419029 + 0.00165211	cv_agg's auc: 0.707597 + 0.0058578
[800]	cv_agg's binary_logloss: 0.418893 + 0.00167332	cv_agg's auc: 0.707694 + 0.00588912
[850]	cv_agg's binary_logloss: 0.4188 + 0.0016772	cv_agg's auc: 0.707736 + 0.00588777
[900]	cv_agg's binary_logloss: 0.418725 + 0.00167827	cv_agg's auc: 0.707861 + 0.00588253
[950]	cv_agg's binary_logloss: 0.418665 + 0.00168126	cv_agg's auc: 0.70795 + 0.00589574
[1000]	cv_agg's binary_logloss: 0.418596 + 0.00168514	cv_agg's auc: 0.708033 + 0.00588002
[1050]	cv_agg's binary_logloss: 0.418558 + 0.00168846	cv_agg's auc: 0.708089 + 0.00589462
[1100]	cv_agg's binary_logloss: 0.418519 + 0.00169371	cv_agg's auc: 0.708094 + 0.00589295
[1150]	cv_agg's binary_logloss: 0.418456 + 0.00169925	cv_agg's auc: 0.70815 + 0.00590788
[1200]	cv_agg's binary_logloss: 0.418441 + 0.00170017	cv_agg's auc: 0.708181 + 0.00590658
[1250]	cv_agg's binary_

[1000]	cv_agg's binary_logloss: 0.418813 + 0.00169603	cv_agg's auc: 0.707248 + 0.00600343
[1050]	cv_agg's binary_logloss: 0.418776 + 0.00169706	cv_agg's auc: 0.707308 + 0.00599947
[1100]	cv_agg's binary_logloss: 0.418736 + 0.0017006	cv_agg's auc: 0.707319 + 0.00600747
[1150]	cv_agg's binary_logloss: 0.418671 + 0.00170422	cv_agg's auc: 0.707393 + 0.00600194
[1200]	cv_agg's binary_logloss: 0.418654 + 0.00170453	cv_agg's auc: 0.707433 + 0.00600354
[1250]	cv_agg's binary_logloss: 0.418612 + 0.00170653	cv_agg's auc: 0.707462 + 0.00599889
[1300]	cv_agg's binary_logloss: 0.41859 + 0.00170631	cv_agg's auc: 0.707483 + 0.00599103
[1350]	cv_agg's binary_logloss: 0.418556 + 0.00170691	cv_agg's auc: 0.707509 + 0.00599264
|  49       | -0.5789   |  0.5      |  0.05     |  0.01     |  550.0    |  0.0      |  0.0      |  100.0    |  5.0      |  16.0     |  20.0     |  0.0      |  0.5      |
[50]	cv_agg's binary_logloss: 0.414964 + 0.00251751	cv_agg's auc: 0.713308 + 0.00724164
[100]	cv_agg's binary_lo

...

[1250]	cv_agg's binary_logloss: 0.418659 + 0.00170761	cv_agg's auc: 0.707492 + 0.00599654
[1300]	cv_agg's binary_logloss: 0.418637 + 0.0017076	cv_agg's auc: 0.7075 + 0.0059935
[1350]	cv_agg's binary_logloss: 0.418604 + 0.0017092	cv_agg's auc: 0.70752 + 0.00599875
|  80       | -0.5793   |  0.5      |  0.05     |  0.01     |  451.9    |  100.0    |  0.0      |  69.52    |  5.0      |  72.54    |  20.0     |  9.723    |  1.0      |
[50]	cv_agg's binary_logloss: 0.442273 + 0.000590525	cv_agg's auc: 0.690885 + 0.00612023
[100]	cv_agg's binary_logloss: 0.434207 + 0.000914114	cv_agg's auc: 0.694917 + 0.00613078
[150]	cv_agg's binary_logloss: 0.429257 + 0.00108928	cv_agg's auc: 0.698441 + 0.00573752
[200]	cv_agg's binary_logloss: 0.425994 + 0.00122265	cv_agg's auc: 0.701215 + 0.00556328
[250]	cv_agg's binary_logloss: 0.423785 + 0.00131847	cv_agg's auc: 0.702906 + 0.00569103
[300]	cv_agg's binary_logloss: 0.422264 + 0.00138545	cv_agg's auc: 0.704453 + 0.00585455
[350]	cv_agg's binary_logloss: 

[150]	cv_agg's binary_logloss: 0.414473 + 0.00273051	cv_agg's auc: 0.714488 + 0.00759118
[200]	cv_agg's binary_logloss: 0.414416 + 0.00273768	cv_agg's auc: 0.714631 + 0.00759469
|  85       | -0.578    |  0.5      |  0.05     |  0.4      |  482.7    |  100.0    |  20.0     |  100.0    |  1.047    |  80.42    |  20.0     |  0.0      |  0.5      |
[50]	cv_agg's binary_logloss: 0.436097 + 0.000794651	cv_agg's auc: 0.720046 + 0.00602704
[100]	cv_agg's binary_logloss: 0.425131 + 0.00123786	cv_agg's auc: 0.723183 + 0.00647305
[150]	cv_agg's binary_logloss: 0.418718 + 0.00157435	cv_agg's auc: 0.726047 + 0.00650188
[200]	cv_agg's binary_logloss: 0.41464 + 0.00185616	cv_agg's auc: 0.727798 + 0.0067725
[250]	cv_agg's binary_logloss: 0.412125 + 0.00206313	cv_agg's auc: 0.7289 + 0.00683049
[300]	cv_agg's binary_logloss: 0.410472 + 0.00222244	cv_agg's auc: 0.72992 + 0.00689563
[350]	cv_agg's binary_logloss: 0.409204 + 0.00241922	cv_agg's auc: 0.730818 + 0.00709376
[400]	cv_agg's binary_logloss: 0.4

[1350]	cv_agg's binary_logloss: 0.414495 + 0.00184114	cv_agg's auc: 0.717448 + 0.00575337
[1400]	cv_agg's binary_logloss: 0.414483 + 0.0018375	cv_agg's auc: 0.717469 + 0.00573335
[1450]	cv_agg's binary_logloss: 0.41446 + 0.00183893	cv_agg's auc: 0.717457 + 0.00573423
|  89       | -0.5858   |  0.5      |  0.05     |  0.01     |  349.4    |  2.146    |  0.4711   |  47.43    |  5.0      |  32.42    |  0.1114   |  20.0     |  1.0      |
[50]	cv_agg's binary_logloss: 0.441694 + 0.000585453	cv_agg's auc: 0.694239 + 0.00544906
[100]	cv_agg's binary_logloss: 0.433661 + 0.000891746	cv_agg's auc: 0.697127 + 0.00545772
[150]	cv_agg's binary_logloss: 0.428835 + 0.00105386	cv_agg's auc: 0.699775 + 0.00533873
[200]	cv_agg's binary_logloss: 0.425778 + 0.00118162	cv_agg's auc: 0.701934 + 0.00534933
[250]	cv_agg's binary_logloss: 0.42382 + 0.00128239	cv_agg's auc: 0.702964 + 0.00552964
[300]	cv_agg's binary_logloss: 0.42254 + 0.00136404	cv_agg's auc: 0.704022 + 0.00570985
[350]	cv_agg's binary_logloss

[600]	cv_agg's binary_logloss: 0.416008 + 0.00176542	cv_agg's auc: 0.715072 + 0.00570797
[650]	cv_agg's binary_logloss: 0.415764 + 0.00178443	cv_agg's auc: 0.715397 + 0.00574207
[700]	cv_agg's binary_logloss: 0.415632 + 0.00179263	cv_agg's auc: 0.715565 + 0.00573981
[750]	cv_agg's binary_logloss: 0.415512 + 0.00179423	cv_agg's auc: 0.715676 + 0.0057363
[800]	cv_agg's binary_logloss: 0.415387 + 0.00180688	cv_agg's auc: 0.715814 + 0.00574334
[850]	cv_agg's binary_logloss: 0.415291 + 0.00181832	cv_agg's auc: 0.715892 + 0.00575585
[900]	cv_agg's binary_logloss: 0.415202 + 0.00182708	cv_agg's auc: 0.716026 + 0.0057557
[950]	cv_agg's binary_logloss: 0.415139 + 0.00182087	cv_agg's auc: 0.716121 + 0.005732
[1000]	cv_agg's binary_logloss: 0.415064 + 0.00182971	cv_agg's auc: 0.716196 + 0.00574198
[1050]	cv_agg's binary_logloss: 0.415025 + 0.00183191	cv_agg's auc: 0.716246 + 0.00574391
[1100]	cv_agg's binary_logloss: 0.414986 + 0.00183141	cv_agg's auc: 0.716252 + 0.0057533
[1150]	cv_agg's binary_

[50]	cv_agg's binary_logloss: 0.418647 + 0.00230433	cv_agg's auc: 0.704207 + 0.00738301
[100]	cv_agg's binary_logloss: 0.418359 + 0.00235586	cv_agg's auc: 0.704864 + 0.00750273
|  103      | -0.581    |  0.7235   |  0.05     |  0.3676   |  328.7    |  0.0      |  0.0      |  4.051    |  5.0      |  16.0     |  20.0     |  20.0     |  0.709    |
[50]	cv_agg's binary_logloss: 0.440985 + 0.00058219	cv_agg's auc: 0.698717 + 0.00549519
[100]	cv_agg's binary_logloss: 0.432261 + 0.000890556	cv_agg's auc: 0.702605 + 0.00566793
[150]	cv_agg's binary_logloss: 0.42695 + 0.0010606	cv_agg's auc: 0.705605 + 0.0055247
[200]	cv_agg's binary_logloss: 0.423486 + 0.0011889	cv_agg's auc: 0.708001 + 0.00549786
[250]	cv_agg's binary_logloss: 0.421261 + 0.00131786	cv_agg's auc: 0.709376 + 0.00556429
[300]	cv_agg's binary_logloss: 0.419837 + 0.00142243	cv_agg's auc: 0.710549 + 0.00570434
[350]	cv_agg's binary_logloss: 0.418713 + 0.001538	cv_agg's auc: 0.711661 + 0.00584317
[400]	cv_agg's binary_logloss: 0.418